# Mapping Cybera Members

In [ ]:
import pandas as pd
#import geopandas
#import pgeocode
#nomi = pgeocode.Nominatim('ca')
import plotly.graph_objects as go
import numpy as np
import folium
try:
    import haversine as hs
except:
    !pip install --user haversine
    import haversine as hs
print('Libraries imported')

## Getting Member Loocations

Using the [Netbox](https://github.com/netbox-community/pynetbox) API, which equires a token, this must be run from on the Cybera network.

In [ ]:
NETBOX_API_KEY = ''

try:
    !pip install --user pynetbox==6.6.2
    import pynetbox
    nb = pynetbox.api('https://netbox.cybera.ca', token=NETBOX_API_KEY)
    sites = nb.dcim.sites
    locations = []
    for site in sites.all():
        locations.append([site, site.latitude, site.longitude])
    df = pd.DataFrame(locations, columns=['site', 'latitude', 'longitude'])
    df.to_csv('data/cybera-members.csv', index=False)
except:
    df = pd.read_csv('data/cybera-members.csv')
display(df)

Drop any NaN values

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)
df

In [ ]:
locations_map = folium.Map(location=[data['Latitude'].mean(), data['Longitude'].mean()], zoom_start=5)
for i in range(0,len(data)):
    folium.Marker([data.iloc[i]['Latitude'], data.iloc[i]['Longitude']], popup=data.iloc[i]['Site']).add_to(locations_map)
locations_map

In [ ]:
df = df.sort_values(by='Latitude')
df.reset_index(inplace=True, drop=True)

distances_list = []
for i in range(len(df)):
    distances_list.append([df.iloc[i]['Site'], df.iloc[i-1]['Site'], 
                      hs.haversine((df.iloc[i]['Latitude'], df.iloc[i]['Longitude']), 
                                   (df.iloc[i-1]['Latitude'], df.iloc[i-1]['Longitude']))])
dbc = pd.DataFrame(distances_list, columns=['Site', 'Next Site', 'Distance'])
dbc

In [ ]:
dbc[dbc['Distance'] < 75]

In [ ]:
cities_to_replace = {
    'Taber':'Lethbridge',
    'Dunmore':'Medicine Hat',
    'Morley':'Canmore',
    'Banff':'Canmore',
    'Ponoka':'Lacombe',
    'Wetaskiwin':'Camrose',
    'Leduc':'Edmonton',
    'Nisku':'Edmonton',
    'Spruce Grove':'Edmonton',
    'Stony Plain':'Edmonton',
    'St. Albert':'Edmonton',
    'Morinville':'Edmonton',
    'Elk Point':'Lac La Biche',
    'Grouard':'High Prairie',
    'Red Earth Creek':'High Prairie',
    'Grimshaw':'Peace River',
}

In [ ]:
rows_to_drop = []
for i in range(len(cities)):
    if cities.iloc[i]['Name'] in cities_to_replace.keys():
        print('drop:', i, cities.iloc[i]['Name'])
        rows_to_drop.append(i)
rows_to_drop

In [ ]:
updated_cities = cities.drop(rows_to_drop, axis=0)
updated_cities.reset_index(inplace=True, drop=True)
updated_cities

In [ ]:
#updated_cities.to_csv('data/updated_cities.csv', index=False)

In [ ]:
replaced_cities = pd.DataFrame.from_dict(cities_to_replace, orient='index', columns=['Replaced'])
replaced_cities.reset_index(inplace=True)

status = pd.read_csv('data/updated_cities.csv')
status.drop(['Latitude', 'Longitude'], axis=1, inplace=True)
for i in range(len(status)):
    city = status.iloc[i]['Name']
    cities = [city]
    if city in replaced_cities['Replaced'].tolist():
        cities.extend(replaced_cities[replaced_cities['Replaced'] == city]['index'].tolist())
    status.loc[i,'Cities'] = str(cities)#.join(';')
    # Count how many members are in or near that "replaced_city"
    #status.loc[i,'Count'] = len(cities)
    member_count = 0
    for city in cities:
        member_count += len(data[data['City'] == city])
    status.loc[i,'Count'] = member_count

status['Count'] = status['Count'].astype(int)
status.drop('Name', axis=1, inplace=True)
status['LED'] = status.index
# set the LED brightness to the number of members in that city (scaled to 100-255)
status['Brightness'] = (status['Count']/status['Count'].max()*155 + 100).astype(int)
status['Red'] = status['Brightness']
status['Green'] = status['Brightness']
status['Blue'] = status['Brightness']
# drop the name column

# export to csv for LEDs
status.drop(['Cities','Count'], axis=1).to_csv('../docs/status.csv', index=False)
status

Create a map for CNC

In [ ]:
updated_cities_map = folium.Map(location=[updated_cities['Latitude'].mean(), updated_cities['Longitude'].mean()], zoom_start=5)
for i in range(0,len(updated_cities)):
    folium.Marker([updated_cities.iloc[i]['Latitude'], updated_cities.iloc[i]['Longitude']], popup=updated_cities.iloc[i]['Name']).add_to(updated_cities_map)
updated_cities_map

In [ ]:
fig0 = go.Figure(data=go.Scattergeo(
    lat = data['Latitude'],
    lon = data['Longitude'],
    text = data['Member']
)
               )

fig0.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig0.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
print('Original member map')
fig0.show()
#fig.write_html('membermap.html')

In [ ]:
fig3 = go.Figure(data=go.Scattergeo(
    lat = updated_cities['Latitude'],
    lon = updated_cities['Longitude'],
    text = updated_cities['Name']
)
               )

fig3.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig3.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
print('Updated member map')
fig3.show()
#fig3.write_html('membermap.html')

In [ ]:
#!pip install kaleido
#fig3.write_image('membermap.svg')

## The order that the LEDs are connected in

In [ ]:
import pandas as pd
import plotly.express as px

updated_cities = pd.read_csv('data/updated_cities.csv')
#px.line(updated_cities.sort_values(by='Latitude'), y='Latitude', x='Longitude', hover_name='Name', height=1000, width=650).update_traces(mode='markers+lines', marker=dict(size=10))
px.line(updated_cities, y='Latitude', x='Longitude', hover_name='Name', height=1000, width=650).update_traces(mode='markers+lines', marker=dict(size=10))

### Reversed mode for labelling the back of the wood

In [ ]:
import pandas as pd
import plotly.express as px

updated_cities = pd.read_csv('data/updated_cities.csv')
updated_cities['Longitude_reversed'] = updated_cities['Longitude'] * -1
fig = px.line(updated_cities, y='Latitude', x='Longitude_reversed', height=1000, width=650)
fig.add_scatter(y=updated_cities['Latitude'], x=updated_cities['Longitude_reversed'], text=updated_cities['Name'], mode='markers+text', textposition='middle right', marker=dict(size=10))
fig.add_scatter(y=updated_cities['Latitude'], x=updated_cities['Longitude_reversed'], text=updated_cities.index, mode='markers+text', textposition='bottom left', marker=dict(size=10))
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(title=None)
fig.update_yaxes(title=None)
fig.update_layout(showlegend=False)
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.show()